In [55]:
!nvidia-smi

Sat Oct 28 20:35:56 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:1B:00.0 Off |                  N/A |
| 31%   38C    P2              57W / 250W |   5755MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import pickle
with open("./data/image_embeddings.pickle", "rb") as f:
  data = pickle.load(f)

In [3]:
len(data.keys())

252

In [4]:
import torch

img_embeddings = []
next_img_embeddings = []
actions = []
instructions = []
for key in data:
  item = data[key]
  img_embeddings.append(item[0])
  next_img_embeddings.append(item[2])
  actions.append(torch.tensor(item[1]).reshape(-1, 2))
  instructions.append(item[3])

img_embeddings_t= torch.cat(img_embeddings, dim=0)
next_img_embeddings_t = torch.cat(next_img_embeddings, dim=0)
actions_t = torch.cat(actions, dim=0)

In [5]:
img_embeddings_t.shape, next_img_embeddings_t.shape, actions_t.shape, len(instructions)

(torch.Size([252, 768]), torch.Size([252, 768]), torch.Size([252, 2]), 252)

In [6]:
# from getpass import getpass
# hf_token = getpass("Huggingface access token")

In [7]:
import bitsandbytes as bnb
import os
import torch

from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig

device_map = {"": 0}

base_model = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = LlamaForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map=device_map,
    use_auth_token=True
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /nfshomes/skarki/miniconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /nfshomes/skarki/miniconda3/envs/llm/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /nfshomes/skarki/miniconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/nfshomes/skarki/miniconda3/envs/llm/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/nfshomes/skarki/miniconda3/envs/llm/lib/libcudart.so'), PosixPath('/nfshomes/skarki/miniconda3/envs/llm/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
tokenizer = LlamaTokenizer.from_pretrained(base_model, use_auth_token=True)
tokenizer.add_special_tokens(
    {
        "pad_token": "<PAD>",
    }
)
# tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
special_tokens_dict = {'additional_special_tokens': ['<IMG>', '<X>', '<Y>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(32004, 4096)

In [9]:
tokenizer.get_added_vocab()

{'<PAD>': 32000, '<IMG>': 32001, '<X>': 32002, '<Y>': 32003}

In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32004, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [11]:
!nvidia-smi

Sat Oct 28 20:35:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:1B:00.0 Off |                  N/A |
| 31%   36C    P2              56W / 250W |   4591MiB / 11264MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    get_peft_config,
    PromptTuningInit,
    PromptTuningConfig,
    TaskType,
    PeftType
)

lora_r = 16
lora_alpha = 32
lora_dropout = 0.1
lora_target_modules = ".*(q_proj|k_proj|v_proj|o_proj|gate_proj|down_proj|up_proj)$"
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 39,976,960 || all params: 6,778,425,344 || trainable%: 0.589767652090261


In [13]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32004, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
              

In [56]:
class ActionSpec:
  def __init__(self, name, low, high, shape):
    self.name = name
    self.low = low
    self.high = high
    self.shape = shape


class ActionTokenizer:
  def __init__(self,
               action_spec: dict,
               vocab_size: int,
               action_order=None):
    self.action_spec = action_spec
    self.vocab_size = vocab_size
    if action_order is None:
      self.action_order = self.action_spec.keys()
    else:
      for action in action_order:
        if action not in self.action_spec.keys():
          raise ValueError(f"actions: {action} not found in action_spec: {action_spec.keys()}")
        assert action in self.action_spec.keys()
      self.action_order = action_order

  def tokenize(self, actions):
    """Tokenizes an action."""
    action_tokens = []
    for k in self.action_order:
      a = actions[k]  # a is [batch, actions_size]
      spec = self.action_spec[k]
      a = torch.clip(a, spec.low, spec.high)
      # Normalize the action [batch, actions_size]
      token = (a - spec.low) / (spec.high - spec.low)
      # Bucket and discretize the action to vocab_size, [batch, actions_size]
      token = (token * (self.vocab_size -1))
      token = token.type(torch.int32)
      action_tokens.append(token)
    # Append all actions, [batch, all_actions_size]
    action_tokens = torch.cat(action_tokens, axis=-1)
    return action_tokens

  def detokenize(self, action_tokens):
    """Detokenizes an action."""
    action = {}
    token_index = 0
    for k in self.action_order:
      spec = self.action_spec[k]
      action_dim = spec.shape[0]
      actions = []
      for _ in range(action_dim):
        spec_index = token_index % action_dim
        a = action_tokens[..., token_index:token_index + 1]
        a = a.type(torch.float32)
        a = a / (self.vocab_size - 1)
        a = (a * (spec.high - spec.low)) + spec.low
        actions.append(a)
        token_index += 1
      action[k] = torch.cat(actions, axis=-1)
    return action


In [15]:
# action_spec = {
#     "dX": torchrl.data.BoundedTensorSpec(low=-10., high=10., shape=(1,), dtype=torch.float32),
#     "dY": torchrl.data.BoundedTensorSpec(low=-10., high=10., shape=(1,), dtype=torch.float32)
# # }
# vocab_size = 1000000
# # action_tokenizer = ActionTokenizer(action_spec, vocab_size)

In [16]:
# action_spec = {
#     "dX": ActionSpec("dX", low=-10., high=10., shape=(1,)),
#     "dY": ActionSpec("dY", low=-10., high=10., shape=(1,))
# }
# vocab_size = 1000000
# action_tokenizer = ActionTokenizer(action_spec, vocab_size)

In [17]:
# import torch
# action = { "dX": torch.Tensor([-0.1]), "dY": torch.Tensor([1.2]) }
# toks = action_tokenizer.tokenize(action)
# toks

In [18]:
# action

In [19]:
# action_tokenizer.detokenize(toks)

In [57]:
from torch.utils.data import Dataset

class LlameDataset(Dataset):
  def __init__(self,
               img_embeddings,
               next_image_embeddings,
               actions,
               instructions,
               tokenizer,
               action_tokenizer):
    self.img_embeddings = img_embeddings
    self.next_image_embeddings = next_image_embeddings
    self.actions = actions
    self.instructions = instructions
    self.tokenizer = tokenizer
    self.action_tokenizer = action_tokenizer

  def __len__(self):
    return self.img_embeddings.shape[0]

  def __getitem__(self, idx):
#     system_prompt = """You are the brain of a robot that can predict what actions the robot should perform next."""
#     prompt = """<s>[INST] <<SYS>>
# {system_prompt}
# <</SYS>>

# {user_message} [/INST]""".format(system_prompt=system_prompt, user_message=user_message)
    img_emb = self.img_embeddings[idx]
    next_img_emb = self.next_image_embeddings[idx]
    act = self.actions[idx]
    actions = { "dX": torch.tensor([act[0].item()]), "dY": torch.tensor([act[1].item()]) }
    dx, dy = self.action_tokenizer.tokenize(actions).tolist()

    inst = self.instructions[idx]
    prefix_1 = """<s>[INST] <<SYS>>
You are the brain of a robot that can predict what actions the robot should perform next.
<</SYS>>

Given """
    prefix_2 = ", what action should the robot take to "
    answer = f"\nAnswer: #X: {dx} #Y: {dy} [/INST]"

    prefix_1_tokens = self.tokenizer(prefix_1, return_tensors="pt", add_special_tokens=False)
    prefix_2_tokens = self.tokenizer(prefix_2, return_tensors="pt", add_special_tokens=False)
    instruction_tokens = self.tokenizer(inst, return_tensors="pt", add_special_tokens=False)
    answer_max_length = 82 - instruction_tokens["input_ids"].shape[1]
    answer_tokens = self.tokenizer(answer, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=answer_max_length)

    
    ins_placeholder = f"{prefix_1}<IMG>{prefix_2}{inst}{answer}"
    placeholder_tokens = prefix_1_tokens["input_ids"].tolist()[0] + [32001] + prefix_2_tokens["input_ids"].tolist()[0]  + instruction_tokens["input_ids"].tolist()[0] + answer_tokens["input_ids"].tolist()[0]
      
    return {
        "current_img_emb": img_emb,
        "next_img_emb": next_img_emb,
        "prefix_1_tokens": prefix_1_tokens,
        "prefix_2_tokens": prefix_2_tokens,
        "instruction_tokens": instruction_tokens,
        "answer_tokens": answer_tokens,
        "placeholder_tokens": torch.tensor(placeholder_tokens),
        "action_x": act[0],
        "action_y": act[1],
        "action_x_token": dx,
        "action_y_token": dy,
        "instruction": inst,
        "placeholder": ins_placeholder
    }

In [58]:
action_spec = {
    "dX": ActionSpec("dX", low=-10., high=10., shape=(1,)),
    "dY": ActionSpec("dY", low=-10., high=10., shape=(1,))
}
vocab_size = 1000000
action_tokenizer = ActionTokenizer(action_spec, vocab_size)

size = 10
ds = LlameDataset(
    img_embeddings_t[:size],
    next_img_embeddings_t[:size],
    actions_t[:size],
    instructions[:size],
    tokenizer,
    action_tokenizer
)

In [22]:
# _x = ds[100]

In [23]:
# print(_x["placeholder"])

In [24]:
# _x["prefix_2_tokens"]["input_ids"]

In [25]:
# _x["instruction_tokens"]["input_ids"]

In [26]:
# _x["answer_tokens"]["input_ids"]

In [27]:
# torch.tensor([[29871,    13, 22550, 29901,   396, 29990, 29901,
#          29871, 29946, 29929, 29929, 29906, 29953, 29900,   396, 29979, 29901,
#          29871, 29945, 29900, 29900, 29896, 29945, 29946,   518, 29914, 25580,
#          29962, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000]])

In [28]:
# _x["prefix_1_tokens"]["input_ids"]

In [29]:
# torch.tensor(_x["placeholder_tokens"]).unsqueeze(0)[:, -82:]

In [30]:
# _x["answer_tokens"]["input_ids"]

In [31]:
# torch.tensor([[2318,   278, 10930,   491,
#           2927,    13, 22550, 29901,   396, 29990, 29901, 29871, 29946, 29929,
#          29929, 29906, 29953, 29900,   396, 29979, 29901, 29871, 29945, 29900,
#          29900, 29896, 29945, 29946,   518, 29914, 25580, 29962, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
#          32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000]])

In [32]:
# tokenizer("[/INST]", return_tensors="pt", add_special_tokens=False)["input_ids"].shape

In [33]:
# tokenizer("\nAnswer: #X: 1000000 #Y: 1000000 [/INST]", return_tensors="pt", add_special_tokens=False)["input_ids"].shape

In [34]:
# tokenizer("""<s>[INST] <<SYS>>
# You are the brain of a robot that can predict what actions the robot should perform next.
# <</SYS>>

# Given <IMG>, what action should the robot take to """, return_tensors="pt")["input_ids"].shape

In [35]:
len(ds)

10

In [59]:
import torch.nn as nn
vit_to_llm = nn.Linear(768, 4096).to(1)

X_head = nn.Sequential(
    nn.Linear(4096, 256),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(256, 1)
).to(1)
Y_head = nn.Sequential(
    nn.Linear(4096, 256),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(256, 1)
).to(1)
next_img_head = nn.Sequential(
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1024, 1024),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1024, 768)
).to(1)

In [37]:
act = actions_t[:10][0]
act

tensor([-0.0003,  0.0175])

In [38]:
actions = { "dX": torch.tensor([act[0].item()]), "dY": torch.tensor([act[1].item()]) }
actions

{'dX': tensor([-0.0003]), 'dY': tensor([0.0175])}

In [39]:
ins_prefix = """<s>[INST] <<SYS>>
You are the brain of a robot that can predict what actions the robot should perform next.
<</SYS>>

Given """
dx, dy = action_tokenizer.tokenize(actions).tolist()
ins_suffix = f", move the red block to the green block. Actions: #X:{dx} #Y:{dy} [/INST]"
ins_prefix_tokens = tokenizer(ins_prefix, return_tensors="pt", add_special_tokens=False)
ins_suffix_tokens = tokenizer(ins_suffix, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=120)

In [40]:
ins_suffix

', move the red block to the green block. Actions: #X:499984 #Y:500875 [/INST]'

In [41]:
ins_placeholder = f"{ins_prefix}<IMG>{ins_suffix}"
ins_tokens = tokenizer(ins_placeholder, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=159)
ins_tokens["input_ids"].shape

torch.Size([1, 159])

In [42]:
ins_placeholder

'<s>[INST] <<SYS>>\nYou are the brain of a robot that can predict what actions the robot should perform next.\n<</SYS>>\n\nGiven <IMG>, move the red block to the green block. Actions: #X:499984 #Y:500875 [/INST]'

In [43]:
ins_prefix_tokens["input_ids"].shape

torch.Size([1, 38])

In [44]:
# ins_suffix_tokens["input_ids"].shape

In [45]:
# ins_prefix_embeddings = model.base_model.model.model.embed_tokens(ins_prefix_tokens["input_ids"].to(0))
# ins_suffix_embeddings =model.base_model.model.model.embed_tokens(ins_suffix_tokens["input_ids"].to(0))

In [46]:
# data_point = ds[0]
# _x = data_point["current_img_emb"]
# _x.shape

In [47]:
# img_embeddings = vit_to_llm(_x.unsqueeze(0).to(1)).to(1)
# img_embeddings.shape

In [48]:
# ins_prefix_embeddings.shape

In [49]:
# ins_suffix_embeddings.shape

In [50]:
# embeddings = torch.cat([ins_prefix_embeddings.to(1), img_embeddings.unsqueeze(0).to(1), ins_suffix_embeddings.to(1)], dim=1)
# embeddings.shape

In [51]:
# labels = ins_tokens["input_ids"]
# labels

In [52]:
# labels.shape

In [53]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    ds,
    batch_size=1,
    shuffle=True,
    num_workers=1
)


In [54]:
# import gc
# # gc.collect()

In [60]:
from tqdm.auto import tqdm
from torch.optim import AdamW
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import torch.nn.functional as F
import random
from torch.autograd import Variable
from itertools import chain
from torch.nn import CrossEntropyLoss

lr = 1e-3

mse_loss = torch.nn.MSELoss()

trainable_models = chain(
    model.parameters(),
    vit_to_llm.parameters(),
    # X_head.parameters(),
    # Y_head.parameters(),
    # next_img_head.parameters()
)

optimizer = AdamW(params=trainable_models, lr=lr)

losses_total = []
losses_vit = []
losses_x = []
losses_y = []
losses_next = []
losses_ce = []

grads_total = []
grads_vit = []
grads_x = []
grads_y = []
grads_next = []

num_epochs = 20
for epoch in tqdm(range(num_epochs)):
  for i, batch in tqdm(enumerate(train_dataloader)):
      prefix_1_embeddings = model.base_model.model.model.embed_tokens(batch["prefix_1_tokens"]["input_ids"].squeeze(1).to(0))
      mapped_img_embeddings = vit_to_llm(batch["current_img_emb"].reshape(batch["current_img_emb"].shape[0], 1, -1).to(1))
      prefix_2_embeddings = model.base_model.model.model.embed_tokens(batch["prefix_2_tokens"]["input_ids"].squeeze(1).to(0))
      instruction_embeddings = model.base_model.model.model.embed_tokens(batch["instruction_tokens"]["input_ids"].squeeze(1).to(0))  
      answer_embeddings = model.base_model.model.model.embed_tokens(batch["answer_tokens"]["input_ids"].squeeze(1).to(0))

      embeddings = torch.cat([
          prefix_1_embeddings.to(0), 
          mapped_img_embeddings.to(0), 
          prefix_2_embeddings.to(0), 
          instruction_embeddings.to(0),
          answer_embeddings.to(0)
      ], dim=1).to(0)
      
      out = model(inputs_embeds=embeddings, output_hidden_states=True)
      

      # Shift so that tokens < n predict n
      shift_logits = out.logits[..., :-1, :].contiguous()
      shift_labels =  batch["placeholder_tokens"].to(0)[..., 1:].contiguous()
      
      shift_logits = shift_logits[:, -82:, :]
      shift_labels = shift_labels[:, -82:]
          
      # Flatten the tokens
      loss_fct = CrossEntropyLoss()
      shift_logits = shift_logits.view(-1, model.config.vocab_size)
      shift_labels = shift_labels.view(-1)
      # Enable model parallelism
      shift_labels = shift_labels.to(shift_logits.device)
      ce_loss = loss_fct(shift_logits, shift_labels)
      loss = ce_loss
      # out = model(inputs_embeds=embeddings, labels=batch["ins_placeholder_tokens"]["input_ids"].to(0), output_hidden_states=True)
      # last_hidden_state = out.hidden_states[-1][:, -1, :].type(torch.float32).to(1)
      # x_out = X_head(last_hidden_state)
      # y_out = Y_head(last_hidden_state)
      # next_img_out = next_img_head(last_hidden_state)

      # ce_loss = out.loss
      # x_loss = mse_loss(batch["action_x"].to("cuda"), x_out)
      # y_loss = mse_loss(batch["action_y"].to("cuda"), y_out)
      # next_img_loss = mse_loss(batch["next_img_emb"].to("cuda"), next_img_out)

      # loss = ce_loss + x_loss + y_loss + next_img_loss
      # loss = ce_loss
      # loss.backward(retain_graph=True)

      # if step % 100 == 0:
      #     G_loss = torch.autograd.grad(loss, W.parameters(), retain_graph=True, create_graph=True)
      #     G_loss_norm = torch.norm(G_loss[0], 2) # G_loss[0] = weights G_loss[1] = bias
      #     G_triplet = torch.autograd.grad(loss_triplet, W.parameters(), retain_graph=True, create_graph=True)
      #     G_triplet_norm = torch.norm(G_triplet[0], 2)
      #     G_ce = torch.autograd.grad(loss_ce, W.parameters(), retain_graph=True, create_graph=True)
      #     G_ce_norm = torch.norm(G_ce[0], 2)

      #     grads_total.append(G_loss_norm.item())
      #     grads_triplets.append(G_triplet_norm.item())
      #     grads_ce.append(G_ce_norm.item())

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      # if step % 100 == 0:
      losses_total.append(loss.item())
      losses_ce.append(ce_loss.item())
      # losses_x.append(x_loss.item())
      # losses_y.append(y_loss.item())
      # losses_next.append(next_img_loss.item())

  avg_total_loss = sum(losses_total) / ((epoch + 1) * len(train_dataloader))
  avg_ce_loss = sum(losses_ce) / ((epoch + 1) * len(train_dataloader))
  # avg_x_loss = sum(losses_x) / ((epoch + 1) * len(train_dataloader))
  # avg_y_loss = sum(losses_y) / ((epoch + 1) * len(train_dataloader))
  # avg_next_loss = sum(losses_next) / ((epoch + 1) * len(train_dataloader))

  print(f"Total: {avg_total_loss}  CE: {avg_ce_loss}")
  # print(f"Total: {avg_total_loss}  CE: {avg_ce_loss} X: {avg_x_loss}  Y: {avg_y_loss} Next: {avg_next_loss}")

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2023-10-28 20:36:36.513950: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-28 20:36:36.565458: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-28 20:36:38.934118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Total: 1.8851272076368333  CE: 1.8851272076368333


0it [00:00, ?it/s]

...

Total: 0.3678657058212492  CE: 0.3678657058212492


0it [00:00, ?it/s]

Total: 0.5236122470349074  CE: 0.5236122470349074


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [62]:
_from = 80
_to = 100
test_ds = LlameDataset(
    img_embeddings_t[_from:_to],
    next_img_embeddings_t[_from:_to],
    actions_t[_from:_to],
    instructions[_from:_to],
    tokenizer,
    action_tokenizer
)

In [63]:
test_dl = DataLoader(
    test_ds,
    batch_size=1,
    num_workers=1
)

In [64]:
_, _d = next(enumerate(test_dl))

In [65]:
_d["instruction"]

['put the blue moon to top left, red moon to top center, yellow star to top right, red pentagon to center left, green cube to center right, green star to bottom left, blue cube to bottom center, yellow pentagon to bottom right, ']

In [72]:
suffix_tokens = tokenizer(""", put the blue moon to top left, red moon to top center, yellow star to top right, red pentagon to center left, green cube to center right, green star to bottom left, blue cube to bottom center, yellow pentagon to bottom right. 
Answer: #X""", return_tensors="pt")
suffix_tokens["input_ids"].squeeze(1)

tensor([[    1,  1919,  1925,   278,  7254, 18786,   304,  2246,  2175, 29892,
          2654, 18786,   304,  2246,  4818, 29892, 13328,  5810,   304,  2246,
          1492, 29892,  2654, 11137, 12841,   304,  4818,  2175, 29892,  7933,
         28704,   304,  4818,  1492, 29892,  7933,  5810,   304,  5970,  2175,
         29892,  7254, 28704,   304,  5970,  4818, 29892, 13328, 11137, 12841,
           304,  5970,  1492, 29889, 29871,    13, 22550, 29901,   396, 29990]])

In [73]:
ans_tokens = tokenizer("\nAnswer: #X:", return_tensors="pt")

In [74]:
_p1_emb = model.base_model.model.model.embed_tokens(_d["prefix_1_tokens"]["input_ids"].squeeze(1).to(0))
_img_emb = vit_to_llm(_d["current_img_emb"].to(1).reshape(_d["current_img_emb"].shape[0], 1, -1).to(1))
_p2_emb = model.base_model.model.model.embed_tokens(_d["prefix_2_tokens"]["input_ids"].squeeze(1).to(0))
_ins_emb = model.base_model.model.model.embed_tokens(_d["instruction_tokens"]["input_ids"].squeeze(1).to(0))
_ans_emb = model.base_model.model.model.embed_tokens(ans_tokens["input_ids"].squeeze(1).to(0))
_merged_emb = torch.cat([_p1_emb.to(0), _img_emb.to(0), _p2_emb.to(0), _ins_emb.to(0), _ans_emb.to(0)], dim=1)
print(_merged_emb.shape)

torch.Size([1, 109, 4096])


In [75]:
_gen = model.generate(
    inputs_embeds=_merged_emb,
    return_dict_in_generate=True,
    output_scores=True,
    output_hidden_states=True,
    max_new_tokens=50
)

In [76]:
hs = _gen.hidden_states
seq = _gen.sequences

In [77]:
seq

tensor([[    1, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000]], device='cuda:0')

In [70]:
tokenizer.pad_token_id

32000

In [78]:
tokenizer.batch_decode(seq, skip_special_tokens=True)

['']

In [48]:
_d["action_x"]

tensor([0.0057])

In [94]:
_d["action_y"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([0.0106])

In [99]:
_next_img_e = _d["next_img_emb"].to("cuda")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [102]:
for i in range(10):
  last_hidden_state = hs[i][-1][:, -1, :]
  _out_x = X_head(last_hidden_state)
  _out_y = Y_head(last_hidden_state)
  _out_next = next_img_head(last_hidden_state)
  print(_out_x)
  print(_out_y)
  print(nn.functional.cosine_similarity(_out_next, _next_img_e))
  print("-" * 10)


tensor([[0.0007]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0100]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.7806], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[-0.0228]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0057]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.7803], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[-0.0135]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0039]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.7765], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[0.0125]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0119]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.7812], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[-0.0310]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0048]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0.7779], device='cuda:0', grad_fn=<SumBackward1>)
----------
tensor([[-0.0015]], device='cud